# Lab11 工人智慧與自駕車(機率)
## Objectives

- 了解電腦影像的處理Pipeline
- 認識Duckiebot辨識路線的流程與方法
- 了解機率統計在實務中的實現與作法

## Tutorials

### Required(先準備好code再看XD)

- [Part1 Lane Filter 作業說明]( https://www.youtube.com/watch?v=dEqg2c0jsRA)
- [Part2 Lane Filter 程式說明](https://www.youtube.com/watch?v=n3s6EvU65jw)
- [Part3 Lane Filter Debug補充](https://www.youtube.com/watch?v=b6Ztd52peoU)
- [Part4 Lane Filter Debug補充2](https://www.youtube.com/watch?v=emrbecL3Ffg)

### Optional

- [Introduction to computer vision](https://classroom.udacity.com/courses/ud810)
  - 可以挑選有興趣的Topic看

## Requirements:

### 安裝需要的新packages(在自己電腦上的python)

```
pip install scipy
```

### 下載Lab11的作業Code到自己的電腦中

```
git clone -b Lab11 https://github.com/GrassLab/NCTU_CS_ZOO.git Lab11
或是手動選擇Lab11的分支下載zip檔
```

之後所有Steps都已經假設你已經SSH連進去小鴨車，並運行好```python -m nodes.duckiebot```

(和HW10/Challenge10一樣，開好就可以不管，且**小鴨車上面跑Lab10已經放過的code即可**，可以不用重傳Lab11的上去)

- 複製原本在Lab10使用的 ```你的電腦某處/Lab10/duckiebot_cs_zoo/app/data/camera_calibration/camera_data.pkl```到```Lab11/duckiebot_cs_zoo/app/data/camera_calibration``` 中 (請自己創建資料夾)

#### 作業相關重要檔案，標記HW Step代表你依序需要開啟(看懂並稍做修改)的部分

- duckiebot_cs_zoo/app/lane 裡面的
  - lane_detector_demo.py: **HW Step1**
  - lane_filter_lr_demo.py: **HW Step3**
  - lane_filter_stop_demo.py: **HW Step4**
- duckiebot_cs_zoo/app裡面的
  - automatic_parking_color.py: **HW Step6**
- duckiebot_cs_zoo/libs裡面的
  - lane_detector.py: **HW Step2**
  - lane_filter_lr.py: Reference for Step3/4
  - lane_filter_stop.py: **HW Step5**

#### 其他方便你用的工具

- Camera Calibration系列遠端版(就是你duckiebot server打開後，直接從你電腦上的python就能做校正了):位於```duckiebot_cs_zoo/app/remote_calib```下面
  - 跑的時候記得，因為有用到DuckiebotClient，環境變數記得設定好(DUCKIE還有PORT)
- ```duckiebot_cs_zoo/utils/video.py```方便你做錄影還有Replay，例如可以先路一段車車行走的影片，再Replay看到的照片，就不用一直去現場跑，用法在程式裡面，可自行理解服用(非作業要求)，最後一次lab可能會用到XD

## 實作流程(請先讀完，了解一下code再開始看TA教學影片
所有Steps都已經假設你SSH進去好小鴨車後```cd Lab10/duckiebot_cs_zoo```，並運行好```python -m nodes.duckiebot```(和HW10/Challenge10一樣，開好就可以不管，小鴨車上面跑Lab10放過的code即可)

**建議將小鴨車攝影機可以看到的環境用厚紙板或貼一些白紙遮一下背景，避免雜物顏色影響路線判斷**

1. HW Step1: 你需要完成**顏色校正與線段偵測**的操作，請參考教學影片進行操作
   - 記得你的```COLOR_RANGE_FILE```名稱，之後會經常用到
   - Challenge11和12你都會需要現場進行調整，尤其是**tune_color_ranges**會需要根據現場光源微調，因此請熟悉這部分操作
   
2. HW Step2: 參考```lane_detector.py```，將 Step1調整出來的參數依序填進去detect_lane函數中，**如果Step1有重做，這步也記得重做**

3. HW Step3: 參考```lane_filter_lr_demo.py```，你只需要填入正確的```COLOR_RANGE_FILE```就能跑起來，可以觀察吐出來的$d$ 和 $\phi$，並可以手動開啟與關閉結果顯示

4. HW Step4: 和Step3相似，開啟```lane_filter_stop_demo.py```後只需填入正確的```COLOR_RANGE_FILE```就能跑起來，但吐出來的```Estimate: dist```都會是一個很怪的定值(因為你還沒開始寫到codeXD)

5. HW Step5: 你要算算**高一數學**還有實作```lane_filter_stop.py```裡面有被

   ```python
   #Start >>> Your code here
   HAHAHA
   #End <<< Your code here
   ```

   包起來的部分，其他地方勿動，共有四處，請參考```Lab11_notes.pdf```

   當你完成後，可以再跑一次Step4的```lane_filter_stop_demo.py```，確定跑起來數值合理

   理論上只會需要用到```numpy```的數學運算，建議先推導好再實作

6. HW Step6: 填入正確的```COLOR_RANGE_FILE```，然後完成```automatic_parking_color.py```，和Lab10 HW一樣，只要大概停到框框中即可，可以思考一下，你有$d,\phi,\text{dist}$後，是否其實就是一個座標系呢？所以停車邏輯基本上和 Lab10 HW是相同的想法

   **這次不用停太完美，能到框中就好**


